# Rugved Task 8-Pointcloud using Open3D

In [37]:
import open3d as o3d
import numpy as np

# Load RGB-D image and create point cloud
def load_point_cloud(rgb_path, depth_path, intrinsic):
    rgb_image = o3d.io.read_image(rgb_path)
    depth_image = o3d.io.read_image(depth_path)

    rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(
        rgb_image, depth_image, depth_scale=1000.0, depth_trunc=3.0, convert_rgb_to_intensity=False
    )

    pcd = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd_image, intrinsic)
    return pcd

# **Slicing: Remove points below a certain height (desk surface)**
def slice_point_cloud(pcd, height_threshold=0.5):
    points = np.asarray(pcd.points)
    colors = np.asarray(pcd.colors)

    mask = points[:, 1] > height_threshold  # Keep points above the threshold
    pcd.points = o3d.utility.Vector3dVector(points[mask])
    pcd.colors = o3d.utility.Vector3dVector(colors[mask])

    return pcd

# **Object Detection using DBSCAN**
def detect_objects(pcd, eps=0.02, min_points=20):
    labels = np.array(pcd.cluster_dbscan(eps=eps, min_points=min_points, print_progress=True))
    
    if labels.size == 0:
        print("⚠️ No clusters found. Try increasing eps or decreasing min_points.")
        return pcd, labels

    max_label = labels.max()
    print(f"✅ Detected {max_label + 1} clusters.")

    # **Assign unique colors per cluster**
    colors = np.random.rand(max_label + 1, 3)  # Generate random colors
    colored_points = np.array(pcd.colors)

    for i in range(len(labels)):
        if labels[i] != -1:  # Ignore noise points (-1)
            colored_points[i] = colors[labels[i]]

    pcd.colors = o3d.utility.Vector3dVector(colored_points)  # Apply new colors

    return pcd, labels

# **Bounding Boxes for Detected Objects**
def add_bounding_boxes(pcd, labels):
    objects = []
    for i in range(labels.max() + 1):  # Iterate over detected clusters
        cluster_indices = np.where(labels == i)[0]
        if len(cluster_indices) == 0:
            continue  # Skip empty clusters

        cluster_pcd = pcd.select_by_index(cluster_indices)  # Extract cluster points
        aabb = cluster_pcd.get_axis_aligned_bounding_box()  # Compute AABB (bounding box)
        aabb.color = (1, 0, 0)  # Red color for bounding boxes
        objects.append(aabb)

    return objects

# **Define camera intrinsic parameters**
intrinsic = o3d.camera.PinholeCameraIntrinsic(width=640, height=480, fx=525.0, fy=525.0, cx=319.5, cy=239.5)

# **Example file paths (replace with your actual file paths)**
rgb_path = r"C:\Users\ashna\desk_2_1.png"
depth_path = r"C:\Users\ashna\desk_2_1_depth.png"

# **Load and process point cloud**
pcd = load_point_cloud(rgb_path, depth_path, intrinsic)
pcd_sliced = slice_point_cloud(pcd, height_threshold=0.5)  # Remove the desk
pcd_with_clusters, labels = detect_objects(pcd_sliced, eps=0.005, min_points=10)

# **Generate Bounding Boxes for Clusters**
bounding_boxes = add_bounding_boxes(pcd_sliced, labels)

# **Visualize: Original vs Processed**
o3d.visualization.draw_geometries([pcd], window_name="Original Point Cloud")
o3d.visualization.draw_geometries([pcd_with_clusters, *bounding_boxes], window_name="Segmented Objects with Bounding Boxes")


✅ Detected 50 clusters.
